### 1. Mount Google Drive
Mounts Google Drive to access the processed Deep Learning (DL) dataset.

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


### 2. Initialize Qdrant Client
Initializes the connection to the Qdrant Cloud vector database. This client will be used to create the `dl_collection` and upload vectors.

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models as rest

qdrant_client = QdrantClient(
     url="Enter your api URL***",
     api_key="Enter your Api key***"
)


### 3. Inspect Processed Data
Reads and prints the first line of the processed DL dataset (`dl.jsonl`) to verify the data format before indexing.

In [4]:
import json

path = "/content/drive/MyDrive/OpenAlex_CS_2025_Data/processed/dl.jsonl"

with open(path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        print(json.loads(line))
        break


{'openalex_id': 'https://openalex.org/W2964015378', 'doi': 'https://doi.org/10.4230/lipics.ecoop.2025.6', 'title': 'Detecting Functionality-Specific Vulnerabilities via Retrieving Individual Functionality-Equivalent APIs in Open-Source Repositories', 'publication_year': 2025, 'publication_date': '2025-01-01', 'authors': [{'author_id': 'https://openalex.org/A5025691113', 'name': 'Thomas Kipf'}, {'author_id': 'https://openalex.org/A5087368991', 'name': 'Max Welling'}], 'concepts': [{'id': 'https://openalex.org/C41008148', 'name': 'Computer science'}, {'id': 'https://openalex.org/C132525143', 'name': 'Graph'}, {'id': 'https://openalex.org/C81363708', 'name': 'Convolutional neural network'}, {'id': 'https://openalex.org/C48044578', 'name': 'Scalability'}, {'id': 'https://openalex.org/C154945302', 'name': 'Artificial intelligence'}, {'id': 'https://openalex.org/C66746571', 'name': 'ENCODE'}, {'id': 'https://openalex.org/C774472', 'name': 'Margin (machine learning)'}, {'id': 'https://openale

### 4. Full Indexing Pipeline (DL Domain)
This is the main ingestion script for the Deep Learning domain:
1.  **Config**: Sets the collection name (`dl_collection`), batch size, and vector size.
2.  **Setup**: Creates the Qdrant collection if it doesn't exist.
3.  **Ingest**:
    *   Reads the `dl.jsonl` file.
    *   Generates embeddings using `all-MiniLM-L6-v2`.
    *   Constructs the payload with metadata.
    *   Uploads points in batches to Qdrant.

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from sentence_transformers import SentenceTransformer
import json
from tqdm import tqdm

# ---------------- CONFIG ----------------
COLLECTION = "dl_collection"   # ✅ NEW COLLECTION
BATCH_SIZE = 64             # safer for cloud
VECTOR_SIZE = 384

# ---------------- MODEL -----------------
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# ---------------- QDRANT ----------------
client = QdrantClient(
   url="Enter your api URL***",
    api_key="Enter your Api key***",
    timeout=60  # ✅ IMPORTANT
)

# ---------------- COLLECTION SETUP ----------------
if not client.collection_exists(COLLECTION):
    client.create_collection(
        collection_name=COLLECTION,
        vectors_config=VectorParams(
            size=VECTOR_SIZE,
            distance=Distance.COSINE
        )
    )

# ---------------- INGEST ----------------
points = []
idx = 0

with open(
    "/content/drive/MyDrive/OpenAlex_CS_2025_Data/processed/dl.jsonl",
    "r",
    encoding="utf-8"
) as f:

    for line in tqdm(f, desc="Indexing DL Papers"):
        paper = json.loads(line)

        text = f"{paper.get('title','')} {paper.get('abstract','')}".strip()
        if len(text) < 30:
            continue

        vector = model.encode(text).tolist()

        payload = {
            "openalex_id": paper.get("openalex_id"),
            "doi": paper.get("doi"),
            "title": paper.get("title"),
            "abstract": paper.get("abstract"),
            "publication_year": paper.get("publication_year"),
            "publication_date": paper.get("publication_date"),
            "venue": paper.get("venue"),
            "citation_count": paper.get("citation_count"),
            "is_open_access": paper.get("is_open_access"),
            "oa_status": paper.get("oa_status"),
            "url": paper.get("url"),

            # 🔥 STRUCTURED FIELDS
            "authors": [
                {
                    "id": a.get("author_id"),
                    "name": a.get("name")
                } for a in paper.get("authors", [])
            ],

            "concepts": [
                {
                    "id": c.get("id"),
                    "name": c.get("name")
                } for c in paper.get("concepts", [])
            ]
        }

        points.append(
            PointStruct(
                id=idx,
                vector=vector,
                payload=payload
            )
        )
        idx += 1

        if len(points) == BATCH_SIZE:
            client.upsert(
                collection_name=COLLECTION,
                points=points,
                wait=False  # ✅ avoids timeout
            )
            points = []

# Final flush
if points:
    client.upsert(
        collection_name=COLLECTION,
        points=points,
        wait=False
    )

print("✅ FULL DL COLLECTION INDEXED SUCCESSFULLY")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Indexing DL Papers: 18751it [07:41, 40.63it/s]


✅ FULL DL COLLECTION INDEXED SUCCESSFULLY


### 5. Test Retrieval (DL Domain)
Performs a sanity check on the indexed `dl_collection`:
1.  Encodes a test query ("What are the open research gaps in deep Learning?").
2.  Searches the collection for the top 5 nearest neighbors.
3.  Prints the detailed results to verify retrieval quality.

In [ ]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

client = QdrantClient(
    url="Enter your api URL***",
     api_key="Enter your Api key***"
)

query = "What are the open research gaps in deep Learning?"
vector = model.encode(query).tolist()

results = client.query_points(
    collection_name="dl_collection" ,
    query=vector,
    limit=5,
    with_payload=True
)

print(f"Returned points: {len(results.points)}")

for i, hit in enumerate(results.points, start=1):
    p = hit.payload

    print(f"\n--- Result {i} ---")
    print("Score:", round(hit.score, 4))
    print("Title:", p.get("title"))
    print("DOI:", p.get("doi"))
    print("Year:", p.get("publication_year"))
    print("Venue:", p.get("venue"))
    print("Citations:", p.get("citation_count"))
    print("Open Access:", p.get("is_open_access"), "| Status:", p.get("oa_status"))
    print("URL:", p.get("url"))

    # ✅ SAFE abstract handling
    abstract = p.get("abstract")
    if abstract:
        print("Abstract:", abstract[:600])
    else:
        print("Abstract: ❌ Not available")

    # ✅ Authors
    authors = p.get("authors", [])
    if authors:
        print("Authors:", ", ".join(a["name"] for a in authors[:5]))
    else:
        print("Authors: ❌ Not available")

    # ✅ Concepts
    concepts = p.get("concepts", [])
    if concepts:
        print("Concepts:", ", ".join(c["name"] for c in concepts[:6]))
    else:
        print("Concepts: ❌ Not available")

Returned points: 5

--- Result 1 ---
Score: 0.6179
Title: Unveiling the frontiers of deep learning: Innovations shaping diverse domains
DOI: https://doi.org/10.1007/s10489-025-06259-x
Year: 2025
Venue: Applied Intelligence
Citations: 3
Open Access: True | Status: hybrid
URL: https://openalex.org/W4408897831
Abstract: ❌ Not available
Authors: Shams Forruque Ahmed, Md. Sakib Bin Alam, Maliha Kabir, Shaila Afrin, Sabiha Jannat Rafa
Concepts: Computer science, Deep learning, Artificial intelligence, Data science

--- Result 2 ---
Score: 0.6035
Title: Evaluating the Trade-offs Between Machine Learning and Deep Learning: A Multi-Dimensional Analysis
DOI: https://doi.org/10.69739/jcsp.v2i1.254
Year: 2025
Venue: Journal of computer, software and program.
Citations: 2
Open Access: True | Status: hybrid
URL: https://openalex.org/W4408195848
Abstract: ❌ Not available
Authors: Nagwa Elmobark
Concepts: Interpretability, Computer science, Artificial intelligence, Machine learning, Implementation, Me